# Lab 01: Sentiment analysis

In [ ]:
import nltk
import numpy as np
import scipy
import tensorflow as tf
from tensorflow import keras

## 1. Data preparation


### 1.1. Data lecture

In [ ]:
import pandas as pd
data = pd.read_csv('finance_sentiment.csv')

data.head(10)

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
5,$SPY wouldn't be surprised to see a green close,positive
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
7,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
8,Kone 's net sales rose by some 14 % year-on-ye...,positive
9,The Stockmann department store will have a tot...,neutral


In [ ]:
from sklearn.model_selection import train_test_split

Xtxt_train, Xtxt_test, Y_train, Y_test = train_test_split(np.array(data["Sentence"]), np.array(data["Sentiment"]), test_size=0.3, random_state=0)
Xtxt_train.shape

(4089,)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
output = label_binarizer.fit_transform(Y_train)

output

array([[0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

### 1.2. Words' encoding

In [ ]:
# Our basic tokenizer using regular expressions
import re

token_pattern = re.compile(r'(?u)\b\w\w+\b')
tokenizer = token_pattern.findall


# Example
tokens = tokenizer('The students are trying to understand this code')
tokens

['The', 'students', 'are', 'trying', 'to', 'understand', 'this', 'code']

In [ ]:
tokenized_sentences = []
for sentence in Xtxt_train:
    tokenized_sentences.append(tokenizer(sentence))

print(tokenized_sentences)

[['maximum', 'of', '666', '104', 'new', 'shares', 'can', 'further', 'be', 'subscribed', 'for', 'by', 'exercising', 'options', 'under', 'the', '2004', 'stock', 'option', 'plan'], ['The', 'sellers', 'were', 'the', 'founders', 'of', 'the', 'company'], ['We', 'are', 'pleased', 'to', 'welcome', 'Tapeks', 'Noma', 'into', 'Cramo', 'group'], ['The', 'company', 'board', 'of', 'directors', 'would', 'propose', 'dividend', 'of', 'EUR0', '15', 'per', 'share', 'for', '2005'], ['The', 'sales', 'of', 'the', 'Tiimari', 'segment', 'fell', 'by', 'year', 'on', 'year', 'to', 'EUR3', 'in', 'June', '2010'], ['YIT', 'CORPORATION', 'SEPT', '24', '2007', 'at', '13', '30', 'CORPORATE', 'RELEASE', 'STOCK', 'EXCHANGE', 'RELEASE', 'YIT', 'CAPITAL', 'MARKETS', 'DAY', 'IN', 'LONDON', 'SEPT', '26', '2007', 'On', 'Wednesday', 'September', '26', '2007', 'YIT', 'will', 'hold', 'Capital', 'Markets', 'Day', 'for', 'investors', 'and', 'analysts', 'in', 'London'], ['Typical', 'end', 'uses', 'include', 'roof', 'structures', '

In [ ]:
from gensim.models import Word2Vec
# TODO: train a Word2Vec model
# ==========================================
model_w2v = Word2Vec(tokenized_sentences, vector_size=15, window=3, min_count=1)

model_w2v

In [ ]:
from gensim.models import FastText
# TODO: train a Fasttext model
# ==========================================
fasttext_model = FastText(tokenized_sentences, vector_size=15, window=5, min_count=1)

fasttext_model

### 1.3. Supporting functions

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
# TODO implement tokenstem
# =============================================
nltk.download('stopwords')

def tokenstem(sentence):
    porter_stemmer = PorterStemmer()
    tokenized_sentence = []
    for token in tokenizer(sentence):
        if not token in nltk.corpus.stopwords.words("english"):
            tokenized_sentence.append(token)
    return [porter_stemmer.stem(word) for word in tokenized_sentence]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def sentvectEncode(wv, sentWords, mx=20):
    centroids = []
    concats = []
    matrices = []

    for sent in sentWords:
        # Initialize vectors for each sentence
        sent_vectors = []

        for word in sent:
            try:
                # Try to encode the word using wv
                word_vect = wv[word]
            except KeyError:
                # If the word is not in the vocabulary, use a zero vector
                word_vect = np.zeros(15)

            sent_vectors.append(word_vect)

        # Centroid encoding
        centroid = np.mean(sent_vectors, axis=0)
        centroids.append(centroid)

        # Concat encoding
        concat = np.concatenate(sent_vectors)[:mx * 15]
        if len(concat) < mx * 15:
            concat = np.pad(concat, (0, mx * 15 - len(concat)), 'constant')
        concats.append(concat)

        # Matrix encoding
        matrix = np.array(sent_vectors[:mx])
        if matrix.shape[0] < mx:
            padding = np.zeros((mx - matrix.shape[0], 15))
            matrix = np.vstack([matrix, padding])
        matrices.append(matrix)

    return np.array(centroids), np.array(concats), np.array(matrices)

### 1.4. Sentence representation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# TODO: TF vectorization
# =============================================
vectorizer = CountVectorizer(tokenizer=tokenstem,max_features=3000)
X_train = vectorizer.fit_transform(Xtxt_train)
X_test = vectorizer.fit_transform(Xtxt_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# TODO: Word2Vec sentences' encodings
# ====================================
w2v_centroids_train, w2v_concat_train, w2v_matrix_train = sentvectEncode(model_w2v.wv, [tokenstem(x) for x in Xtxt_train])
w2v_centroids_test, w2v_concat_test, w2v_matrix_test = sentvectEncode(model_w2v.wv, [tokenstem(x) for x in Xtxt_test])

In [ ]:
# TODO: Fasttext sentences' encodings
# ====================================
fasttext_centroids, fasttext_concat, fasttext_matrix = sentvectEncode(fasttext_model.wv, [tokenstem(x) for x in Xtxt_train])
fasttext_centroids_test, fasttext_concat_test, fasttext_matrix_test = sentvectEncode(fasttext_model.wv, [tokenstem(x) for x in Xtxt_test])

## 2. Training models

### 2.1. TF-based/MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# ==========================================


# Instantiate and train the model
TF_MNB = MultinomialNB()
TF_MNB.fit(X_train, Y_train)  # Assuming Y_train is not one-hot encode



MultinomialNB()

### 2.2. Victor-based/MLP

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
# TODO: design a function which creates a model with
# an input followed by a dense of 5 neurones/relu followed by an output layer
# =====================================================================

def create_mlp_model(input_dim, output_dim):
    input_layer = Input(shape=(input_dim,))
    dense_layer = Dense(5, activation='relu')(input_layer)
    output_layer = Dense(output_dim, activation='softmax')(dense_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Instantiate models for each encoding
output_dim = 3  # Number of unique classes

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
def create_mlp_model2(input_dim, output_dim):
    input_layer = Input(shape=(input_dim,))

    # Add more dense layers
    dense_layer1 = Dense(128, activation='relu')(input_layer)  # First dense layer with 128 neurons
    dense_layer2 = Dense(64, activation='relu')(dense_layer1)  # Second dense layer with 64 neurons
    dense_layer3 = Dense(32, activation='relu')(dense_layer2)  # Third dense layer with 32 neurons

    # Output layer
    output_layer = Dense(output_dim, activation='softmax')(dense_layer3)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Instantiate the model for each encoding
output_dim = 3  # Adjust this based on the number of unique classes in your dataset
input_dim = 100  # Adjust this based on the input dimension of your data

mlp_model = create_mlp_model(input_dim, output_dim)

# Model Summary
mlp_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 5)                 505       
                                                                 
 dense_1 (Dense)             (None, 3)                 18        
                                                                 
Total params: 523 (2.04 KB)
Trainable params: 523 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# W2VCent_MLP word2vec centroid with MLP
# =============================================
W2VCent_MLP = create_mlp_model2(15, output_dim)

W2VCent_MLP.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 15)]              0         
                                                                 
 dense_2 (Dense)             (None, 128)               2048      
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 12483 (48.76 KB)
Trainable params: 12483 (48.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# TODO: W2VCent_MLP word2vec centroid with MLP training
# =============================================
W2VCent_MLP.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
W2VCent_MLP.fit(w2v_centroids_train, output, epochs=100)

Epoch 1/100
128/128 [==============================] - 9s 6ms/step - loss: 1.0012 - accuracy: 0.5280
Epoch 2/100
128/128 [==============================] - 1s 5ms/step - loss: 0.9468 - accuracy: 0.5564
Epoch 3/100
128/128 [==============================] - 1s 4ms/step - loss: 0.9305 - accuracy: 0.5598
Epoch 4/100
128/128 [==============================] - 0s 4ms/step - loss: 0.9249 - accuracy: 0.5564
Epoch 5/100
128/128 [==============================] - 0s 3ms/step - loss: 0.9226 - accuracy: 0.5600
Epoch 6/100
128/128 [==============================] - 0s 4ms/step - loss: 0.9171 - accuracy: 0.5595
Epoch 7/100
128/128 [==============================] - 1s 4ms/step - loss: 0.9175 - accuracy: 0.5591
Epoch 8/100
128/128 [==============================] - 1s 5ms/step - loss: 0.9172 - accuracy: 0.5635
Epoch 9/100
128/128 [==============================] - 1s 5ms/step - loss: 0.9110 - accuracy: 0.5627
Epoch 10/100
128/128 [==============================] - 1s 5ms/step - loss: 0.9087 - accura

In [ ]:
# TODO: W2VConcat_MLP word2vec concatenation with MLP
# =============================================
W2VConcat_MLP = create_mlp_model(300, output_dim)
W2VConcat_MLP.summary()

In [ ]:
# TODO: W2VConcat_MLP word2vec centroid with MLP training
# =================================================
W2VConcat_MLP.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
W2VConcat_MLP.fit(w2v_concat_train, output, epochs=100)

In [ ]:
# TODO: W2VCent_MLP fasttext centroid with MLP
# =============================================
FTCent_MLP = create_mlp_model(5, output_dim)
FTCent_MLP.summary()

In [ ]:
# TODO: W2VCent_MLP fasttext centroid with MLP training
# =============================================
FTCent_MLP.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
FTCent_MLP.fit(fasttext_centroids, output, epochs=100)

In [ ]:
# TODO: FTConcat_MLP fasttext concatenation with MLP
# =============================================
FTConcat_MLP = create_mlp_model(100, output_dim)
FTConcat_MLP.summary()

In [ ]:
# TODO: FTConcat_MLP fasttext concatenation with MLP training
# =============================================
FTConcat_MLP.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
FTConcat_MLP.fit(fasttext_concat, output, epochs=100)

### 2.3. Matrix-based/CNN

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten
from tensorflow.keras.models import Model
# TODO: design a function which creates a model with
# an input followed by a conv1d (5 filters, 3 window) followed by a maxpool (strides=1)
# followed by a flatten, then a dense (output)
# =====================================================================
def create_cnn_model(input_shape, output_dim):
    # Define the input layer
    input_layer = Input(shape=input_shape)

    # Add a Conv1D layer
    conv_layer = Conv1D(filters=5, kernel_size=3, activation='relu')(input_layer)

    # Add a MaxPool1D layer
    pool_layer = MaxPool1D(strides=1)(conv_layer)

    # Add a Flatten layer
    flatten_layer = Flatten()(pool_layer)

    # Add a Dense (output) layer
    output_layer = Dense(output_dim, activation='softmax')(flatten_layer)  # Use 'sigmoid' for binary classification

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Example Usage
output_dim = 3  # Replace with the number of classes in your dataset
input_shape = (20, 5)  # Replace with the appropriate input shape

In [ ]:
# TODO: W2VMat_CNN word2vec matrix with CNN
# =============================================
W2VMat_CNN = create_cnn_model(input_shape, output_dim)
W2VMat_CNN.summary()

In [ ]:
# TODO: W2VMat_CNN word2vec matrix with CNN training
# =============================================
W2VMat_CNN.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
W2VMat_CNN.fit(w2v_matrix_train, output, epochs=100)

In [ ]:
# TODO: WFTMat_CNN fasttext matrix with CNN
# =============================================
FTMat_CNN = create_cnn_model(input_shape, output_dim)
FTMat_CNN.summary()

In [ ]:
# TODO: WFTMat_CNN fasttext matrix with CNN training
# =============================================
FTMat_CNN.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
FTMat_CNN.fit(fasttext_matrix, output, epochs=100)

## 3. Testing models

In [ ]:
import timeit
from sklearn.metrics import classification_report

# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = TF_MNB.predict(X_test)
print('m1', timeit.default_timer()-t)

print(classification_report(Y_test, Ypred, zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = W2VCent_MLP.predict(w2v_centroids_test)
print('W2VCent_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = W2VConcat_MLP.predict(w2v_concat_test)
print('W2VConcat_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = FTCent_MLP.predict(fasttext_centroids_test)
print('FTCent_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = FTConcat_MLP.predict(fasttext_concat_test)
print('FTConcat_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = W2VMat_CNN.predict(w2v_matrix_test)
print('FTConcat_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
# TODO: complete
# ==============
t = timeit.default_timer()
Ypred = FTMat_CNN.predict(fasttext_matrix_test)
print('FTConcat_MLP', timeit.default_timer()-t)

print(classification_report(Y_test, label_binarizer.inverse_transform(Ypred), zero_division=0))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming the number of features in your word2vec concatenated data is set in 'input_dim'
input_dim = w2v_concat_train.shape[1]  # Replace with actual shape
output_dim = 3  # Replace with the number of output classes

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(input_dim, 1)))  # '50' is the number of LSTM units
model.add(Dense(output_dim, activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM layer
X_train_reshaped = w2v_concat_train.reshape((w2v_concat_train.shape[0], input_dim, 1))

# Train the model
model.fit(X_train_reshaped, Y_train, epochs=10, batch_size=32)  # Replace Y_train with your labels and adjust epochs, batch_size as needed

# Model Summary
model.summary()


NameError: ignored

In [ ]:
Y_train

In [ ]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load BERT tokenizer and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize your data (example text)
input_text = list(Xtxt_train)
tokenized_text = tokenizer(input_text, padding=True, truncation=True, return_tensors="tf")

# Define the TensorFlow model
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# BERT outputs
outputs = bert_model(input_ids, attention_mask=attention_mask)
pooled_output = outputs.pooler_output

# Add a classification layer
hidden_layer = tf.keras.layers.Dense(10, activation='relu')(pooled_output)
classifier_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)  # Change to softmax for multi-class

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=classifier_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# model.fit(...)  # You need to prepare your data similar to tokenized_text

model.summary()


In [ ]:
# Example training data
train_texts = list(Xtxt_train)  # Replace with your training texts
train_labels = [0, 1, ...]  # Replace with your training labels (0 or 1 for binary classification)

# Tokenize training data
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="tf")
train_labels = tf.convert_to_tensor(train_labels)

# Training the model
model.fit(
    [train_encodings['input_ids'], train_encodings['attention_mask']],
    train_labels,
    epochs=2,  # Number of epochs
    batch_size=8  # Batch size
)

# Example testing data
test_texts = ["Test text 1", "Test text 2", ...]  # Replace with your testing texts
test_labels = [0, 1, ...]  # Replace with your testing labels

# Tokenize testing data
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors="tf")
test_labels = tf.convert_to_tensor(test_labels)

# Evaluating the model
evaluation = model.evaluate(
    [test_encodings['input_ids'], test_encodings['attention_mask']],
    test_labels
)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])